<a href="https://colab.research.google.com/github/O-Kpy/Dacon/blob/main/Dacon_%E1%84%82%E1%85%B2%E1%84%89%E1%85%B3_%E1%84%90%E1%85%A9%E1%84%91%E1%85%B5%E1%86%A8_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2_AI_%E1%84%80%E1%85%A7%E1%86%BC%E1%84%8C%E1%85%B5%E1%86%AB%E1%84%83%E1%85%A2%E1%84%92%E1%85%AC_%E1%84%82%E1%85%B2%E1%84%89%E1%85%B3_%E1%84%90%E1%85%A9%E1%84%91%E1%85%B5%E1%86%A8_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2_AI_%E1%84%80%E1%85%A7%E1%86%BC%E1%84%8C%E1%85%B5%E1%86%AB%E1%84%83%E1%85%A2%E1%84%92%E1%85%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!pip install transformers
!pip install sentencepiece

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  32379      0 --:--:-- --:--:-- --:--:-- 32379
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.9.0+cu102
     |████████████████████████████████| 57 kB 2.6 MB/s 
Uninstalling torch-1.9.0+cu102:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.272 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatibl

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch_xla
import torch_xla.core.xla_model as xm

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold

import transformers
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import XLMRobertaConfig, XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from transformers import AdamW

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/뉴스 토픽 분류 AI 경진대회/뉴스 토픽 분류 AI 경진대회_train_data.csv', index_col='index')
test = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/뉴스 토픽 분류 AI 경진대회/뉴스 토픽 분류 AI 경진대회_test_data.csv', index_col='index')
submit = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/뉴스 토픽 분류 AI 경진대회/뉴스 토픽 분류 AI 경진대회_sample_submission.csv')
topic_dict = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/뉴스 토픽 분류 AI 경진대회/뉴스 토픽 분류 AI 경진대회_topic_dict.csv')

In [4]:
df = shuffle(train)
sk = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

y_train = df['topic_idx']

fold_list = list(sk.split(df, y_train))

train_list = []
val_list = []

for i, fold in enumerate(fold_list):
  df_train = df[df.index.isin(fold[0])]
  df_val = df[df.index.isin(fold[1])]

  train_list.append(df_train)
  val_list.append(df_val)

print(f'train length : {len(train_list)}')
print(f'val length : {len(val_list)}')

train length : 5
val length : 5


In [8]:
MODEL_TYPE = 'xlm-roberta-base'

NUM_FOLDS = 5

# Saving 5 TPU models will exceed the 4.9GB disk space.
# Therefore, will will only train on 3 folds.
NUM_FOLDS_TO_TRAIN = 3 

L_RATE = 3e-6
MAX_LEN = 256
NUM_EPOCHS = 3
BATCH_SIZE = 32

device = xm.xla_device()
print(device)

xla:1


In [6]:
# bert-base-multilingual-uncased 토큰
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE,
                                          do_lower_case=True)

class CompDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'title']

    encoded_dict = tokenizer.encode_plus(sentence1,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    target = torch.tensor(self.df_data.loc[index, 'topic_idx'])

    sample = (input_ids, attention_mask, target)
    return sample

  def __len__(self):
    return len(self.df_data)


class TestDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'title']

    encoded_dict = tokenizer.encode_plus(sentence1,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    padded_input_ids = encoded_dict['input_ids'][0]
    mask = encoded_dict['attention_mask'][0]
    
    sample = (padded_input_ids, mask)
    
    return sample

  def __len__(self):
    return len(self.df_data)

In [7]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
test = test.reset_index()

In [9]:
train = CompDataset(df_train)
valid = CompDataset(df_val)
test_set = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=4)
val_dataloader = torch.utils.data.DataLoader(valid,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_set,
                                               batch_size=BATCH_SIZE,
                                               shuffle=False,
                                               num_workers=4)

In [28]:
for i,j,k in train_dataloader:
  print(i,j,k)
  break

tensor([[     0,  34781,  27681,  ...,      1,      1,      1],
        [     0,      6, 111071,  ...,      1,      1,      1],
        [     0,  25284, 105040,  ...,      1,      1,      1],
        ...,
        [     0,   7063,   3776,  ...,      1,      1,      1],
        [     0, 110177,      6,  ...,      1,      1,      1],
        [     0,  28700,   7084,  ...,      1,      1,      1]]) tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]) tensor([1, 6, 1, 0, 2, 6, 5, 3, 4, 0, 5, 5, 0, 2, 4, 0, 6, 1, 0, 4, 4, 2, 2, 1,
        3, 5, 4, 5, 4, 4, 4, 6])


In [29]:
for i,j in test_dataloader:
  print(i,j)
  break

tensor([[     0,   5241, 135903,  ...,      1,      1,      1],
        [     0,  14327,  15164,  ...,      1,      1,      1],
        [     0,   5920, 159539,  ...,      1,      1,      1],
        ...,
        [     0,  17882,  80524,  ...,      1,      1,      1],
        [     0, 203559,  57326,  ...,      1,      1,      1],
        [     0,  27141,   7901,  ...,      1,      1,      1]]) tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [10]:
# model 불러오기(BertForSequnceClassification)
model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_TYPE,
                                                      num_labels=y_train.nunique(),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [11]:
import random
import gc

seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

fold_val_acc_list = []
for i in range(0, NUM_FOLDS):
  fold_val_acc_list.append([])

# for each fold
for epoch in range(0, NUM_EPOCHS):
  print("\nNum folds used for training:", NUM_FOLDS_TO_TRAIN)
  print(f'=========== Epoch {epoch+1} / {NUM_EPOCHS} ===========')

  num_folds = len(train_list)

  epoch_acc_scores_list = []

  for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
    print('\n== Fold Model', fold_index)

    # Training
    stacked_val_labels=[]
    targets_list=[]
    optimizer = AdamW(model.parameters(), lr=L_RATE)

    print('Training...')
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss=0

    for i, batch in enumerate(train_dataloader):
      train_status = 'Batch' + str(i+1) + 'of' + str(len(train_dataloader))

      print(train_status, end='\r')

      b_input_ids = batch[0].to(device)
      b_attention_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      model.zero_grad()

      outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
      
      loss=outputs[0]
      total_train_loss = total_train_loss + loss.item()
      
      optimizer.zero_grad()
      loss.backward()
      
      # Clip the norm of the gradients to 1.0.
      # This is to help prevent the "exploding gradients" problem.
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Optimizer for GPU
      # opimizer.step()

      # Optimizer for TPU
      xm.optimizer_step(optimizer=optimizer, barrier=True)

    print('Train loss:', total_train_loss)

      
    # Validation
    print('\nValidation...')
    model.eval()
      
    torch.set_grad_enabled(False)
    total_val_loss=0

    for j, val_batch in enumerate(val_dataloader):
      val_status = 'Batch' + str(j+1) + 'of' + str(len(val_dataloader))
      print(val_status, end='\r')

      input_ids = val_batch[0].to(device)
      attention_mask = val_batch[1].to(device)
      labels = val_batch[2].to(device)

      outputs = model(input_ids=input_ids,
                      attention_mask=attention_mask,
                      labels=labels)
        
      loss = outputs[0]
      total_val_loss = total_val_loss + loss.item()
        
      preds = outputs[1]
      val_preds = preds.detach().cpu().numpy()
      target_np = labels.to('cpu').numpy()

      targets_list.extend(target_np)

      if j == 0:  # first batch
        stacked_val_preds = val_preds
      else:
        stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

      
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)

    val_acc = accuracy_score(y_true, y_pred)

    epoch_acc_scores_list.append(val_acc)

    print('Val Loss:', total_val_loss)
    print('Val Acc:', val_acc)

    # save the best model
    if epoch == 0:
      model_name = 'model_' + str(fold_index) + '.bin'
      torch.save(model.state_dict(), model_name)
      print('Saved model as ', model_name)

    if epoch != 0:
      val_acc_list = fold_val_acc_list[fold_index]
      best_val_acc = max(val_acc_list)

      if val_acc > best_val_acc:
        model_name = 'model' + str(fold_index) + '.bin'
        torch.save(model.state_dict(), model_name)
        print('Val acc improved. Saved model as ', model_name)
      
    # save the best fold
    fold_val_acc_list[fold_index].append(val_acc)
    gc.collect()  # save memory

  

  cv_acc = sum(epoch_acc_scores_list) / NUM_FOLDS_TO_TRAIN
  print('\nCV Acc:', cv_acc)


Num folds used for training: 3
=========== Epoch 1 / 3 ===========

== Fold Model 0
Training...
Train loss: 869668.1601936817

Validation...
Val Loss: 246532.84844970703
Val Acc: 0.10492880613362542
Saved model as  model_0.bin

== Fold Model 1
Training...
Train loss: 861316.646408081

Validation...
Val Loss: 143413.19775390625
Val Acc: 0.16801752464403066
Saved model as  model_1.bin

== Fold Model 2
Training...
Train loss: 826583.5635375977

Validation...
Val Loss: 141128.58587646484
Val Acc: 0.1592552026286966
Saved model as  model_2.bin

CV Acc: 0.14406717780211756

Num folds used for training: 3
=========== Epoch 2 / 3 ===========

== Fold Model 0
Training...


KeyboardInterrupt: ignored

In [13]:
print('\nTest Set...')

model_preds_list = []

print('Total batches:', len(test_dataloader))

for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
    
    print('\nFold Model', fold_index)

    # Load the fold model
    path_model = 'model_' + str(fold_index) + '.bin'
    model.load_state_dict(torch.load(path_model))  # load model

    # Send the model to the device.
    model.to(device)


    stacked_val_labels = []
    

    # Put the model in evaluation mode.
    model.eval()

    # Turn off the gradient calculations.
    # This tells the model not to compute or store gradients.
    # This step saves memory and speeds up validation.
    torch.set_grad_enabled(False)


    # Reset the total loss for this epoch.
    total_val_loss = 0

    for j, h_batch in enumerate(test_dataloader):

        inference_status = 'Batch ' + str(j + 1)

        print(inference_status, end='\r')

        b_input_ids = h_batch[0].to(device)
        b_input_mask = h_batch[1].to(device)


        outputs = model(b_input_ids,
                        attention_mask=b_input_mask)


        # Get the preds
        preds = outputs[0]


        # Move preds to the CPU
        val_preds = preds.detach().cpu().numpy()
        
        
        # Stack the predictions.

        if j == 0:  # first batch
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        
    model_preds_list.append(stacked_val_preds)
    
            
print('\nPrediction complete.')  


Test Set...
Total batches: 286

Fold Model 0

Fold Model 1

Fold Model 2

Prediction complete.


In [15]:
preds = np.argmax(stacked_val_preds, axis=1)

submit = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/대회/뉴스 토픽 분류 AI 경진대회/submit1.csv')
submit['topic_idx'] = preds
submit

array([0, 3, 2, ..., 2, 4, 2])

In [20]:
submit.to_csv('/content/drive/MyDrive/dataset/Dacon/대회/뉴스 토픽 분류 AI 경진대회/submit1.csv', index=False)